# 蓝绿部署基本操作以及思路
* 基础设施：
    * 云平台：AWS，阿里云等
    * 流量监控：TrafficManager
    * API管理：Axway
* 前期准备：以AWS作为云平台
    * AMI-AI（AWS申请资源的ID），workspace（系统所属工作空间），nexusCredential（上传系统代码需要nenux账号），stashCredential / githubCredential（拉代码的授权），awsCredential（操作AWS的授权）
    * jenkins pipeline
    * 部署机器配置好对应的AWS命令行工具，python类库，boto3，cloud-accounts-client，awscli
    * 基础公共操作类库，比如版本号生成器，调用真正执行AWS操作的接口
    * 基础AWS操作类库，包括ASG创建，ELB创建，Attach ASG TO ELB， Dettach ASG FROM ELB, 删除ASG等
* 部署前后端不分离的APP，或者纯前端APP：
    * 在同一个workspace下面，创建一些必要的stack，比如系统全局监控stack，邮件通知stack等
    * 创建snapshot，该步骤主要是配置基础设施软件，如tomcat，jvm等，并将部署脚本等基础性的脚本打包准备好，类似于创建一个docker镜像
        * 如果需要使用HTTPS，在这一步要配置jvm的证书，同时也要配置tomcat的接口
        * 部署脚本一般就是去第三方，比如nexus，下载项目app和配置文件，然后启动项目
    * 给每个环境创建ELB，如SYS，UAT，PROD
        * 准备好ELB的模板，AWS上面有参考
        * 将监控器加入到ELB中
        * 在workspace下面导入相关证书，然后在模板中使用，可以通过HTTPS访问ELB
        * 确定ELB的stack name，这个名字要注册到TrafficManager上面，确保外网能够访问
        * 注册访问ELB的端口，一般是80和443
        * 定义转发接口，也就是配置在ASG中app的接口，ELB需要将请求过来的80或433接口转发到真实app接口
    * 把ELB注册到TrafficManager上面
        * 在TrafficManager创建新的F5 pool，然后把ELB加到pool的member里面
    * 为每个环境创建ASG，如SYS，UAT，PROD
        * 准备好ASG的模板，AWS上面有参考
        * 使用上面准备好的snapshot，将其绑定到ASG中
        * 执行部署脚本，下载项目代码和配置文件，启动系统，并发邮件通知ASG机器创建成功
        * 将新的ASG attach到green ELB上面做PVT
        * 将新的ASG从green ELB上面dettach，然后attach到blue ELB上面
        * 删除老的ASG
* 部署后端APP，也就是API：
    * 在同一个workspace下面，创建一些必要的stack，比如系统全局监控stack，邮件通知stack等
    * 创建snapshot，该步骤主要是配置基础设施软件，如tomcat，jvm等，并将部署脚本等基础性的脚本打包准备好，类似于创建一个docker镜像
        * 如果需要使用HTTPS，在这一步要配置jvm的证书，同时也要配置tomcat的接口
        * 部署脚本一般就是去第三方，比如nexus，下载项目app和配置文件，然后启动项目
    * 创建带有ELB的ASG
        * 在创建ASG的同时，创建好ELB，AWS上面有对应的模板
        * 将ELB的DNS名字以及自己定义的green url发给Axway，在Axway上面注册green api，做PVT
        * 将Axway上关于该API注册的所有url都删除，包括blue的url
        * 将ELB的DNS名字以及自己定义的blue url发给Axway，在Axway上面注册blue api
        * 删除老的ASG和对应的ELB